Соревнование по структуризации чеков: https://ods.ai/competitions/nlp-receipts.
Базовое, быстрое решение (менее 1 минуты на домашнем ноутбуке). Только с ипользованием регулярных выражений и простой логики, без использования алгоритмов ML и нейронный сетей. Скор: 0,5982225063 (94 место). Общедоступное базовое решение с использованием NER: 0,4857637482.

In [49]:
from gensim.models.fasttext import FastText
import pandas as pd
import numpy as np
import re
import string

from sklearn.model_selection import train_test_split
import torch

from collections import Counter


In [50]:
train=pd.read_csv('train_supervised_dataset.csv').fillna('')
train['name']=train.name.str.lower()
#train

In [51]:
test=pd.read_csv('test_dataset.csv').fillna('')
test['name']=test.name.str.lower()
#test

In [52]:
train

,id,name,good,brand
0,0,petmax бантик леопард с красн розой 2шт,бантик,petmax
1,1,87191 бусы для елки шарики_87191,бусы,
2,2,футболка piazza italia wr011446881,футболка,piazza italia
3,3,7) yi572-03x-one заколка для волос для девочки,заколка,
4,4,одежда (вес) 1500,одежда,
...,...,...,...,...
24995,24995,"вода ""саирме"" с/г 500мл",вода,sairme
24996,24996,моя семя . 0.175л и ассортим,,моя семья
24997,24997,рулет бисквитн.яшкино клубничный со слив,рулет,яшкино
24998,24998,460075794371 почвогрунт цветочное счастье фас...,почвогрунт,фаско


In [53]:
#Замена символов в функции разбита на элементы для более удобной отладки.

def preprocess_text(text):
    if type(text) == float:
        print(text)    
    #text = re.sub(r"[0-9]+", r"9", text) #Replace all numbers with 9
    text = re.sub(r"[/]", r' ', text)
    text = re.sub(r"[0-9]+", r" ", text) #Replace all numbers with 9
    text = re.sub(r"[\\]", r' ', text)
    text = re.sub(r"[.]", r' ', text)
    text = re.sub(r"[_%]", r' ', text)
    #text = re.sub(r'[,:;!%#?_]', r' ', text)
    text = re.sub(r'[,:;]', r' ', text)
    text = re.sub(r'["]', r'', text)
    text = re.sub(r"[']", r'', text)
    text = re.sub(r"[' ']+", r' ', text)
    text = re.sub(r"[*]+", r'', text)
    text = re.sub(r"[)]", r'', text)
    text = re.sub(r"[(]+", r'', text)
    text = re.sub(r"[]]+", r'', text)
    text = re.sub(r"[[]+", r'', text)
    text = re.sub(r"[!?]", r'', text)
    text = re.sub(r"[-]", r' ', text)
    text = re.sub(r"[-]", r' ', text)
    text = re.sub(r"[+]", r' ', text)
    return text

In [54]:
train['name']=train.name.apply(preprocess_text)
test['name']=test.name.apply(preprocess_text)

In [55]:
train['tokens']=train.name.str.split()
test['tokens']=test.name.str.split()

In [56]:
#train.head()

Определим функцию, которая будет на выделенном для обучения фрейме данных формировать соотвествие товаров брендам.

In [57]:
def good_brand(d:pd.DataFrame()):
    g_b=d.groupby('good')['brand'].agg([('brand','unique')]).reset_index()
    return g_b

In [58]:
t=good_brand(train)
t.head(3)

,good,brand
0,,"[био баланс, gerber, becherovka, alpen gold, л..."
1,absent,[xenta]
2,ale,[ginger ale]


In [59]:
max_good=train.good.apply(lambda x: len(x.split(' '))).max()
max_brand=train.brand.apply(lambda x: len(x.split(' '))).max()
max_name=train.name.apply(lambda x: len(x.split(' '))).max()
max_good, max_brand, max_name

(6, 5, 51)

Определяем общий словарь.

In [60]:
class Vocabulary(object):
    """Класс для обработки обучающей выборки и и формирования общего словаря по текстам name, good, brand."""

    def __init__(self, token_to_idx=None):
        """
        Args:
            token_to_idx (dict): существущий словарь соответствия токенов индексам.
        """

        if token_to_idx is None:
            token_to_idx = {'<UNK>': 1} #Маркер для не включенных в словарь слов.
        self._token_to_idx = token_to_idx

        self._idx_to_token = {idx: token 
                              for token, idx in self._token_to_idx.items()}
        
    def to_serializable(self):
        """ Возвращает словарь, который может быть сериализован """
        return {'token_to_idx': self._token_to_idx}

    @classmethod
    def from_serializable(cls, contents):
        """ создает экземпляр Vocabulary из сериализованного словаря"""
        return cls(**contents)

    def add_token(self, token):
        """Обновление словаря на основе токена.

        Args:
            token (str): Токен для добавления в словарь.
        Returns:
            index (int): Число соответствующее токену.
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)+1
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
            
    def add_many(self, tokens):
        """Добавление списка токенов в словарь.
        
        Args:
            tokens (list): Лист токенов.
        Returns:
            indices (list): Лист индексов, соответсвующих токену.
        """
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token):
        
        return self._token_to_idx.get(token, 1) #Если слова нет в словаре возврaщает индекс не включенного в словарь слова.
        

    def lookup_index(self, index):
        """
        Raises:
            KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]
   
    def __len__(self):
        return len(self._token_to_idx)

In [61]:
class Vectorizer(object):
    
    def __init__(self, all_vocab):
        self.all_vocab = all_vocab #Общий словарь созданный на основе тренировой выборки.
                        
    def vectorize_text(self, text, vector_length=max_name):
        """
        Args:
            text (str): Строка текста, разделенная на слова по пробелам
            vector_length (int): Аргумент, для принудительного изменения длины вектора. 
        Returns:
            the vetorized text (numpy.array)
        """
        indices = []
        for token in text.split(" "):
            if (token not in string.punctuation) and (token !=' ') and (token !=''):            
                indices.append(self.all_vocab.lookup_token(token))
        
        if vector_length < 0:
            vector_length = len(indices)

        out_vector = np.zeros(vector_length, dtype=np.int64)
        out_vector[:len(indices)] = indices
        out_vector[len(indices):] = 0 #Заполняем вектор нулями до максимальной длины. 

        return out_vector
    
    def index_good(self, check_df):
        """"
            Args: check_df: обучающий дата сет.
            Returns: Список с номерами индексов из общего словаря, для токенов, содержащихся в описании товаров.
        """
        indices=[]
        for i in range (len(check_df)):
            line=check_df.iloc[i]
            good_list=line.good.split(' ')
            for w in good_list:
                if (w not in string.punctuation) and (w !=' ') and ((w !='')): 
                    indices.append(self.all_vocab.lookup_token(w))
                    w_r1=re.sub(r'["]', r'', w)
                    w_r2 = re.sub(r"[']", r'', w)
                    indices.append(self.all_vocab.lookup_token(w_r1))
                    indices.append(self.all_vocab.lookup_token(w_r2))
        return list(set(indices))  
    
    
    def index_brand(self, check_df):
        indices=[]
        for i in range (len(check_df)):
            line=check_df.iloc[i]
            brand_list=line.brand.split(' ')
            for w in brand_list:
                if (w not in string.punctuation) and (w !=' ') and ((w !='')): 
                    indices.append(self.all_vocab.lookup_token(w))
                    w_r1=re.sub(r'["]', r'', w)
                    w_r2 = re.sub(r"[']", r'', w)
                    indices.append(self.all_vocab.lookup_token(w_r1))
                    indices.append(self.all_vocab.lookup_token(w_r2))
        return list(set(indices))
                
        
    def good_from_tokens(self, good_index, tokens):
        """"
            Args: 
            good_index: Список индексов всех товаров из обучающей выборки
            tokens: Список слов (из поля name) из валидацинной или тренировочной выборки
            
            Returns: 
            '' в случае если, в векторном представлении name не найдено индексов товаров
            good - товар из всего списка товаров, найденный в строке name.
        """
        good=''
        count=0
        for token in tokens:            
            ind=self.all_vocab.lookup_token(token)
            if (ind in good_index) and (ind!=1):
                count+=1
                good=good+token+' '
                
        good=good.strip()  
        good=good.split()
        if len(good)>0:
            good=good[0]#Мы предсказываем только товары, состоящие из одного слова! 
                        
        #if count!=1 or len(good)<4:
            #good=''
        if len(good)<4:#Исключаем возможные предлоги и т. д., которые тоже могут быть в описании товара
            good=''
        
        return good
    
    def brand_from_tokens(self, brand_index, good_index, tokens):
        brand=''
        count=0
        for token in tokens:            
            ind=self.all_vocab.lookup_token(token)
            if (ind in brand_index) and (ind!=1) and (ind not in good_index) and (len(token)>2):
            #if (ind in brand_index) and (ind!=1)  and (len(token)>2):
                count+=1
                brand=brand+token+' '
                        
        brand=brand.strip()
        #if count!=1 or len(brand)<4: #Предсказываем только брэнды, состоящие из одного описания
        #if count!=1:
            #brand=''
        #if len (brand)>0:
            #brand=brand.split()[0]
        
        return brand
        
        
     
    @classmethod
    def from_dataframe(cls, train_df, cutoff=0):
                
        word_counts = Counter()
                        
        for i in range(len(train_df)): 
            line=train_df.iloc[i]
            text=[]
            text.extend(line.tokens)
            text.extend(line.good.split(' '))
            text.extend(line.brand.split(' '))
                                    
            for token in text:
                                  
                if (token not in string.punctuation) and (token !=' ') and ((token !='')):                 
                    word_counts[token] += 1
        
        all_vocab = Vocabulary()
        for word, word_count in word_counts.items():
            if word_count >= cutoff:
                all_vocab.add_token(word)
                
                
        return cls(all_vocab)

    @classmethod
    def from_serializable(cls, contents):
        all_vocab =Vocabulary.from_serializable(contents['all_vocab'])
                
        return cls(all_vocab=all_vocab)

    def to_serializable(self):
        return {'all_vocab': self.all_vocab.to_serializable()}
                

In [62]:
all_df=pd.concat([train, test], axis=0, ignore_index=True).fillna('')
all_df

,id,name,good,brand,tokens
0,0,petmax бантик леопард с красн розой шт,бантик,petmax,"[petmax, бантик, леопард, с, красн, розой, шт]"
1,1,бусы для елки шарики,бусы,,"[бусы, для, елки, шарики]"
2,2,футболка piazza italia wr,футболка,piazza italia,"[футболка, piazza, italia, wr]"
3,3,yi x one заколка для волос для девочки,заколка,,"[yi, x, one, заколка, для, волос, для, девочки]"
4,4,одежда вес,одежда,,"[одежда, вес]"
...,...,...,...,...,...
29995,4995,рамка п сл кость,,,"[рамка, п, сл, кость]"
29996,4996,энерг напиток red bull л,,,"[энерг, напиток, red, bull, л]"
29997,4997,наконечники т никель шт,,,"[наконечники, т, никель, шт]"
29998,4998,шоколад риттерспорт мол с цел миндалем г,,,"[шоколад, риттерспорт, мол, с, цел, миндалем, г]"


In [63]:
train_df, val_df = train_test_split(train, test_size=len(train)//3, shuffle=True, random_state=32)
vect=Vectorizer.from_dataframe(all_df)
good_index=vect.index_good(train_df)
brand_index=vect.index_brand(train_df)
vect.all_vocab.__len__(), len(good_index), len(brand_index)


(33277, 2370, 5722)

In [64]:
vect.all_vocab.lookup_token('опора')

1123

In [65]:
#vect.all_vocab.to_serializable()

In [66]:
len(val_df)

8333

Предсказываем по первому алгоритму товары и брэнды.

In [67]:
val_df['good_pred']=val_df.tokens.apply(lambda x: vect.good_from_tokens(good_index, x))
val_df['brand_pred']=val_df.tokens.apply(lambda x: vect.brand_from_tokens(brand_index, good_index, x))

Находим соответствие брэндов товарам:

In [68]:
good_brand=train_df.groupby('good')['brand'].agg([('brand_list','unique')]).reset_index()
good_brand.rename(columns={'good': 'good_for_brand'}, inplace=True)
good_brand['brand_list']=good_brand.brand_list.apply(lambda x: list(x))
good_brand.head(3)

,good_for_brand,brand_list
0,,"[d&m, hayabusa, tide, педигри, nescafe, pulpy,..."
1,absent,[xenta]
2,bra,[nike]


In [69]:
#brand_len=good_brand.brand_list.apply(lambda x: len(x))


Добавим в валидационный фрейм столбец со списком брэндов, соответствующих каждой товарной позиции.

In [70]:
pred_all=pd.merge(val_df, good_brand, how='left', left_on='good_pred', right_on='good_for_brand').\
drop(['good_for_brand'], axis=1)
pred_all['brand_pred']=pred_all.brand_pred.str.split(' ')#Преобразуем в список для следующего алгоритма.
pred_all.head(30)

,id,name,good,brand,tokens,good_pred,brand_pred,brand_list
0,20725,котовник кошачий базилио ц п,котовник,базилио,"[котовник, кошачий, базилио, ц, п]",,[],"[d&m, hayabusa, tide, педигри, nescafe, pulpy,..."
1,6724,аэрозоль д волос y,аэрозоль,,"[аэрозоль, д, волос, y]",аэрозоль,[],"[decorix, gardex, , chameleon, olvist, пантено..."
2,13044,молоко бирское бзмж тфа шт,молоко,бирское,"[молоко, бирское, бзмж, тфа, шт]",молоко,[],"[белый медведь, агуша, рогачевъ, свитлогорье, ..."
3,3966,спагетти саргуль копченый г алт сыры,спагетти,саргуль,"[спагетти, саргуль, копченый, г, алт, сыры]",спагетти,"[копченый, сыры]","[delverde, алтайская сказка, шебекинские, ]"
4,9133,творог маруся г,творог,маруся,"[творог, маруся, г]",творог,[],"[, первый вкус, сваля, растишка, золотые луга,..."
5,19815,вафли глазир орех изюм вес кдв яшкино,вафли,яшкино,"[вафли, глазир, орех, изюм, вес, кдв, яшкино]",вафли,"[кдв, яшкино]","[кдв, иванян, сладонеж, varfetto, яшкино, , ак..."
6,3780,эл перс испаритель hqd cuvie plus черника,испаритель,hqd,"[эл, перс, испаритель, hqd, cuvie, plus, черника]",испаритель,[plus],"[puff bar plus, cricket]"
7,955,булочка улыбка с клубничным повидлом кг нг,булочка,улыбка,"[булочка, улыбка, с, клубничным, повидлом, кг,...",булочка,[улыбка],"[, навашинский хлеб, сказка, корона]"
8,951,рулетка победит м,рулетка,победит,"[рулетка, победит, м]",рулетка,[],"[флекси, flexi, richmann, classic]"
9,16337,пиво светлое непастеризованное жигулевское л с...,пиво,жигулевское,"[пиво, светлое, непастеризованное, жигулевское...",пиво,[жигулевское],"[, балтика, халзан, охота, афанасий, heineken,..."


In [71]:
pred_all.isnull().sum()

id              0
name            0
good            0
brand           0
tokens          0
good_pred       0
brand_pred      0
brand_list    132
dtype: int64

In [72]:
pred_all['brand_list']=pred_all['brand_list'].fillna(pred_all.brand_pred)

Реализуем алгоритм предсказания товарных брэндов по уже предсказанным товарам: В таблице pred_all сравниваем токены со столбцом brand_all. Выделяем общие слова (токены) и выводим их в качестве предсказания в той последовательности, которая была в поле brand_all. Если соответствия нет - выводим пустую строку.

In [73]:
def pred_brands(tokens, brands):
    all_words=list(set(tokens)&set(brands))
    brand=''
    if len(all_words)>0:
        for b in brands:
            if (b in all_words) and len(b)>2:
                brand=brand+' '+b
    brand=brand.strip()    
    return brand

In [74]:
pred_all['brand_pred_2']=pred_all.apply(lambda x: pred_brands(x.tokens, x.brand_list), axis=1)

Соединяем два предсказания брэндов в одно: Если нет предсказания brand_pred_2 (более точное) - заполняем предсказанием brand_pred_1.

In [75]:
def pred_connect(x1, x2):
    x_out=x2
    if len(x2)==0 and len(x1)>0:
        x_out=x1[0]
    return(x_out)

In [76]:
pred_all['brand_pred_all']=pred_all.apply(lambda x: pred_connect(x.brand_pred, x.brand_pred_2), axis=1)


In [77]:
#Удаляем далее не нужные столбцы.
pred_all.drop(['name', 'tokens', 'brand_list', 'brand_pred', 'brand_pred_2'], axis=1, inplace=True)
#pred_all.drop(['name', 'tokens', 'brand_list'], axis=1, inplace=True)
pred_all

,id,good,brand,good_pred,brand_pred_all
0,20725,котовник,базилио,,
1,6724,аэрозоль,,аэрозоль,
2,13044,молоко,бирское,молоко,
3,3966,спагетти,саргуль,спагетти,копченый
4,9133,творог,маруся,творог,
...,...,...,...,...,...
8328,11273,лампа,feron,лампа,feron
8329,14993,слойка,,слойка,
8330,23457,зубная щетка,oral-b,зубная,
8331,21626,йогурт,золотая нива,йогурт,черная


Посчитаем точность.

In [78]:
def count_acc(x1, x2):
    return x1==x2

In [79]:
pred_all['match_good']=pred_all.apply(lambda x: count_acc(x.good, x.good_pred), axis=1)
pred_all['match_brand']=pred_all.apply(lambda x: count_acc(x.brand, x.brand_pred_all), axis=1)

In [80]:
pred_all.match_good.sum()/len(pred_all), pred_all.match_brand.sum()/len(pred_all)

(0.7922716908676347, 0.5721828873154926)

Выполним предсказание на тестовой выборке.

In [81]:
vect=Vectorizer.from_dataframe(train)
good_index=vect.index_good(train)
brand_index=vect.index_brand(train)
vect.all_vocab.__len__(), len(good_index), len(brand_index)

(29558, 2857, 7482)

In [82]:
test['good_pred']=test.tokens.apply(lambda x: vect.good_from_tokens(good_index, x))
test['brand_pred']=test.tokens.apply(lambda x: vect.brand_from_tokens(brand_index, good_index, x))

In [83]:
len_good=test.good_pred.apply(lambda x: len(x))
(len_good>0).sum()/len(test)

0.808

In [84]:
len_brand=test.brand_pred.apply(lambda x: len(x))
(len_brand>0).sum()/len(test)

0.4612

In [85]:
good_brand=train.groupby('good')['brand'].agg([('brand_list','unique')]).reset_index()
good_brand.rename(columns={'good': 'good_for_brand'}, inplace=True)
good_brand.head(3)

,good_for_brand,brand_list
0,,"[био баланс, gerber, becherovka, alpen gold, л..."
1,absent,[xenta]
2,ale,[ginger ale]


In [86]:
pred_all=pd.merge(test, good_brand, how='left', left_on='good_pred', right_on='good_for_brand').\
drop(['good_for_brand'], axis=1)
pred_all['brand_pred']=pred_all.brand_pred.str.split(' ')#Преобразуем в список для следующего алгоритма.
pred_all.head(10)

,id,name,tokens,good_pred,brand_pred,brand_list
0,0,ермак клей универсальный мл блистер,"[ермак, клей, универсальный, мл, блистер]",клей,[ермак],"[экон, tytan, , metylan, zvezda, хомакол, kleo..."
1,1,торт сладушка зимняя вишня г,"[торт, сладушка, зимняя, вишня, г]",торт,[зимняя],"[наш кондитер, , даханаго, faretti, пекарь, хл..."
2,2,смеситель calorie а д кухни,"[смеситель, calorie, а, д, кухни]",смеситель,[кухни],"[frap, blanco, fmark, juguni, , iddis, bach, l..."
3,3,лимон гр бар,"[лимон, гр, бар]",,[],"[био баланс, gerber, becherovka, alpen gold, л..."
4,4,коньяк сараджишвили лет л грузия,"[коньяк, сараджишвили, лет, л, грузия]",коньяк,[],"[лезгинка, бастион, дюльбер, henri mounier, ко..."
5,5,born pretty пластина для стемпинга bp l textur...,"[born, pretty, пластина, для, стемпинга, bp, l...",пластина,[арт],"[, parisnail, bistrong]"
6,6,рис chicken beer с гуляшом гр,"[рис, chicken, beer, с, гуляшом, гр]",,[beer],"[био баланс, gerber, becherovka, alpen gold, л..."
7,7,труба tebo п п d pn м стекловолокно хлыст,"[труба, tebo, п, п, d, pn, м, стекловолокно, х...",труба,[tebo],"[, lammin, kalde, pro aqua, htem, политрон, va..."
8,8,грандорф консервы для собак куропатка с индейк...,"[грандорф, консервы, для, собак, куропатка, с,...",консервы,[],"[almo nature, маримолоко, animonda, , one&only..."
9,9,|одеяло автотепло №,"[|одеяло, автотепло, №]",,[],"[био баланс, gerber, becherovka, alpen gold, л..."


In [87]:
pred_all['brand_list']=pred_all['brand_list'].fillna(pred_all.brand_pred)

In [88]:
pred_all['brand_pred_2']=pred_all.apply(lambda x: pred_brands(x.tokens, x.brand_list), axis=1)
len_brand_2=pred_all.brand_pred_2.apply(lambda x: len(x))
(len_brand_2>0).sum()/len(pred_all)

0.0692

In [89]:
pred_all['brand_pred_all']=pred_all.apply(lambda x: pred_connect(x.brand_pred, x.brand_pred_2), axis=1)
pred_all.isnull().sum()

id                0
name              0
tokens            0
good_pred         0
brand_pred        0
brand_list        0
brand_pred_2      0
brand_pred_all    0
dtype: int64

In [90]:
pred_all=pred_all.drop(['name', 'tokens', 'brand_list', 'brand_pred_2', 'brand_pred'], axis=1)
pred_all.rename(columns={'good_pred': 'good', 'brand_pred_all': 'brand'}, inplace=True)
pred_all

,id,good,brand
0,0,клей,ермак
1,1,торт,зимняя
2,2,смеситель,кухни
3,3,,
4,4,коньяк,
...,...,...,...
4995,4995,рамка,
4996,4996,напиток,red
4997,4997,,
4998,4998,шоколад,


In [91]:
pred_all.isnull().sum()

id       0
good     0
brand    0
dtype: int64

In [92]:
pred_all.to_csv('subm_re.csv', index=False)
pd.read_csv('subm_re.csv').fillna('').tail(20)

,id,good,brand
4980,4980,,
4981,4981,масло,zic
4982,4982,рубашка,lee
4983,4983,печенье,
4984,4984,фреза,fit
4985,4985,лампочка,ergolux
4986,4986,фруктовый,
4987,4987,спрей,аквалор
4988,4988,крем,
4989,4989,штопор,
